In [22]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [51]:
%set_env OPENAI_API_KEY=
%set_env TAVILY_API_KEY=

env: OPENAI_API_KEY=
env: TAVILY_API_KEY=


In [32]:
from uuid import uuid4
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage

def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    # assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    # merge the new messages with the existing messages
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # replace any existing messages with the same id
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # append any new messages to the end
            merged.append(message)
    return merged
    
class AgentState(TypedDict):
    messages:Annotated[list[AnyMessage],reduce_messages]

In [33]:
tool = TavilySearchResults()

In [34]:
class Agent:
    def __init__(self, model, tools, system="", checkpointer=None):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=checkpointer,
            interrupt_before=["action"]
        )
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        print(state)
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [35]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-3.5-turbo")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [36]:
messages = [HumanMessage(content="what is the weather in Hyderabad, Telangana")]
thread = {"configurable":{"thread_id":"1"}}
for event in abot.graph.stream({"messages":messages},thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='what is the weather in Hyderabad, Telangana', id='d452428d-821b-4c9f-9696-d10f63d21abf'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_i3miRtuwzbN40DJCkr0iFwgE', 'function': {'arguments': '{"query":"weather in Hyderabad, Telangana"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 156, 'total_tokens': 180}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-256302f9-a7bc-42bf-b173-9230f9af69e1-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Hyderabad, Telangana'}, 'id': 'call_i3miRtuwzbN40DJCkr0iFwgE'}])]}
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_i3miRtuwzbN40DJCkr0iFwgE', 'function': {'arguments': '{"query":"weather in Hyderabad, Telangana"}', 'name': 'tavily_search_results_json'}, 'type': 'fu

In [37]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='what is the weather in Hyderabad, Telangana', id='d452428d-821b-4c9f-9696-d10f63d21abf'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_i3miRtuwzbN40DJCkr0iFwgE', 'function': {'arguments': '{"query":"weather in Hyderabad, Telangana"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 156, 'total_tokens': 180}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-256302f9-a7bc-42bf-b173-9230f9af69e1-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Hyderabad, Telangana'}, 'id': 'call_i3miRtuwzbN40DJCkr0iFwgE'}])]}, next=('action',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24af2-7d84-6078-8005-b41aaf0aeb0f'}}, metadata={'source': 'loop', 'step': 5, 'writes': {'llm': {'messages': [AIMessage(content='', ad

In [38]:
abot.graph.get_state(thread).next

('action',)

In [39]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Hyderabad, Telangana'}, 'id': 'call_i3miRtuwzbN40DJCkr0iFwgE'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Hyderabad\', \'region\': \'Sindh\', \'country\': \'Pakistan\', \'lat\': 25.37, \'lon\': 68.37, \'tz_id\': \'Asia/Karachi\', \'localtime_epoch\': 1717752080, \'localtime\': \'2024-06-07 14:21\'}, \'current\': {\'last_updated_epoch\': 1717751700, \'last_updated\': \'2024-06-07 14:15\', \'temp_c\': 40.2, \'temp_f\': 104.4, \'is_day\': 1, \'condition\': {\'text\': \'Sunny\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/113.png\', \'code\': 1000}, \'wind_mph\': 19.7, \'wind_kph\': 31.7, \'wind_degree\': 232, \'wind_dir\': \'SW\', \'pressure_mb\': 1001.0, \'pressure_in\': 29.56, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 33, \'cloud\': 0, \'feelslike_c\': 46.0, \'feelslike_f\': 114.7, \'windchill_c\

In [40]:
abot.graph.get_state(thread).next

()

In [41]:
messages = [HumanMessage("Whats the weather in Munagala Village, Itikyala Mandal?")]
thread = {"configurable": {"thread_id": "3"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in Munagala Village, Itikyala Mandal?', id='35318fd9-b6cd-45d6-88f3-6b8280551384'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_RPTTiBvr2IH7Go6gw2uIzR74', 'function': {'arguments': '{"query":"weather in Munagala Village, Itikyala Mandal"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 161, 'total_tokens': 190}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-25b9fbc9-46dc-452a-a49f-fa365aae15b5-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Munagala Village, Itikyala Mandal'}, 'id': 'call_RPTTiBvr2IH7Go6gw2uIzR74'}])]}
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_RPTTiBvr2IH7Go6gw2uIzR74', 'function': {'arguments': '{"query":"weather in Munagala Village, Itikyala Mandal"}',

In [42]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in Munagala Village, Itikyala Mandal?', id='35318fd9-b6cd-45d6-88f3-6b8280551384'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_RPTTiBvr2IH7Go6gw2uIzR74', 'function': {'arguments': '{"query":"weather in Munagala Village, Itikyala Mandal"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 161, 'total_tokens': 190}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-25b9fbc9-46dc-452a-a49f-fa365aae15b5-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Munagala Village, Itikyala Mandal'}, 'id': 'call_RPTTiBvr2IH7Go6gw2uIzR74'}])]}, next=('action',), config={'configurable': {'thread_id': '3', 'thread_ts': '1ef24b05-5803-6e3a-8001-76eb213b5d7a'}}, metadata={'source': 'loop', 'step': 1, 'writes': {'llm': 

In [43]:
abot.graph.get_state(thread).next

('action',)

In [44]:
current_values = abot.graph.get_state(thread)

In [46]:
current_values.values['messages'][-1].tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'weather in Munagala Village, Itikyala Mandal'},
  'id': 'call_RPTTiBvr2IH7Go6gw2uIzR74'}]

In [47]:
_id = current_values.values['messages'][-1].tool_calls[0]['id']
current_values.values['messages'][-1].tool_calls = [{'name': 'tavily_search_results_json',
  'args': {'query': 'weather in Alampur, Telangana'},
  'id': _id}]

In [48]:
abot.graph.update_state(thread,current_values.values)

{'messages': [HumanMessage(content='Whats the weather in Munagala Village, Itikyala Mandal?', id='35318fd9-b6cd-45d6-88f3-6b8280551384'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_RPTTiBvr2IH7Go6gw2uIzR74', 'function': {'arguments': '{"query":"weather in Munagala Village, Itikyala Mandal"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 161, 'total_tokens': 190}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-25b9fbc9-46dc-452a-a49f-fa365aae15b5-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Alampur, Telangana'}, 'id': 'call_RPTTiBvr2IH7Go6gw2uIzR74'}])]}


{'configurable': {'thread_id': '3',
  'thread_ts': '1ef24b10-cc79-6ff2-8002-64108f57756c'}}

In [49]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in Munagala Village, Itikyala Mandal?', id='35318fd9-b6cd-45d6-88f3-6b8280551384'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_RPTTiBvr2IH7Go6gw2uIzR74', 'function': {'arguments': '{"query":"weather in Munagala Village, Itikyala Mandal"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 161, 'total_tokens': 190}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-25b9fbc9-46dc-452a-a49f-fa365aae15b5-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Alampur, Telangana'}, 'id': 'call_RPTTiBvr2IH7Go6gw2uIzR74'}])]}, next=('action',), config={'configurable': {'thread_id': '3', 'thread_ts': '1ef24b10-cc79-6ff2-8002-64108f57756c'}}, metadata={'source': 'update', 'step': 2, 'writes': {'llm': {'messages': 

In [50]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Alampur, Telangana'}, 'id': 'call_RPTTiBvr2IH7Go6gw2uIzR74'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Alampur\', \'region\': \'\', \'country\': \'Bangladesh\', \'lat\': 24.87, \'lon\': 88.27, \'tz_id\': \'Asia/Dhaka\', \'localtime_epoch\': 1717752819, \'localtime\': \'2024-06-07 15:33\'}, \'current\': {\'last_updated_epoch\': 1717752600, \'last_updated\': \'2024-06-07 15:30\', \'temp_c\': 42.8, \'temp_f\': 109.0, \'is_day\': 1, \'condition\': {\'text\': \'Sunny\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/113.png\', \'code\': 1000}, \'wind_mph\': 6.9, \'wind_kph\': 11.2, \'wind_degree\': 354, \'wind_dir\': \'N\', \'pressure_mb\': 997.0, \'pressure_in\': 29.43, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 19, \'cloud\': 12, \'feelslike_c\': 44.3, \'feelslike_f\': 111.7, \'windchill_c\': 42.8, \'